In [20]:
from emukit.test_functions.multi_fidelity import (multi_fidelity_park_function,multi_fidelity_borehole_function)
from emukit.examples.multi_fidelity_dgp.baseline_model_wrappers import LinearAutoRegressiveModel

import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [21]:
from collections import namedtuple

Function = namedtuple('Function', ['name', 'y_scale', 'noise_level', 'do_x_scaling', 'num_data', 'fcn'])

park = Function(name='park', y_scale=1, noise_level=[0., 0.], do_x_scaling=False, num_data=[30, 5], 
                    fcn=multi_fidelity_park_function)

borehole = Function(name='borehole', y_scale=100, noise_level=[0.05, 0.1], do_x_scaling=True, num_data=[60, 5], 
                    fcn=multi_fidelity_borehole_function)

In [22]:
def generate_data(fcn_tuple, n_test_points):
    """
    Generates train and test data for
    """
    
    # A different definition of the parameter space for the branin function was used in the paper
    if fcn_tuple.name == 'branin':
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace([ContinuousParameter('x1', -5., 0.), ContinuousParameter('x2', 10., 15.)])
    else:
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace(space._parameters[:-1])
    
    do_x_scaling = fcn_tuple.do_x_scaling
    
    
    # Generate training data
    
    latin = LatinDesign(new_space)
    X = [latin.get_samples(n) for n in fcn_tuple.num_data]
    
    # Scale X if required
    if do_x_scaling:
        scalings = X[0].std(axis=0)
    else:
        scalings = np.ones(X[0].shape[1])
        
    for x in X:
        x /= scalings
    
    Y = []
    for i, x in enumerate(X):
        Y.append(fcn.f[i](x * scalings))
    
    y_scale = fcn_tuple.y_scale
    
    # scale y and add noise if required
    noise_levels = fcn_tuple.noise_level
    if any([n > 0 for n in noise_levels]):
        for y, std_noise in zip(Y, noise_levels):
            y /= y_scale + std_noise * np.random.randn(y.shape[0], 1)
    
    # Generate test data
    x_test = latin.get_samples(n_test_points)
    x_test /= scalings
    y_test = fcn.f[-1](x_test * scalings)
    y_test /= y_scale

    i_highest_fidelity = (len(fcn_tuple.num_data) - 1) * np.ones((x_test.shape[0], 1))
    x_test = np.concatenate([x_test, i_highest_fidelity], axis=1)
    print(X[1].shape)
    return x_test, y_test, X, Y

In [23]:
def calculate_metrics(y_test, y_mean_prediction, y_var_prediction):
    # R2
    r2 = r2_score(y_test, y_mean_prediction)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_mean_prediction))
    # Test log likelihood
    mnll = -np.sum(scipy.stats.norm.logpdf(y_test, loc=y_mean_prediction, scale=np.sqrt(y_var_prediction)))/len(y_test)
    return {'r2': r2, 'rmse': rmse, 'mnll': mnll}

In [24]:
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.core.initial_designs import LatinDesign

In [25]:
np.random.seed(123)

x_test, y_test, X, Y = generate_data(park, 1000)

x_test_b, y_test_b, Xb, Yb = generate_data(borehole, 1000)

#m1 =  LinearAutoRegressiveModel(X, Y)
#m1.optimize()

#print(X[0])

(5, 4)
(5, 8)


In [26]:
import scipy.stats

#print(calculate_metrics(y_test, y_mean, y_var))

In [27]:
import matplotlib.pyplot as plt

#plt.scatter(y_mean,y_test)
#plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
#plt.xlim([0,55])
#plt.ylim([0,55])
#plt.show()

In [28]:
#from emukit.examples.multi_fidelity_dgp.multi_fidelity_deep_gp import MultiFidelityDeepGP

In [29]:
#mf_dgp_fix_lf_mean = MultiFidelityDeepGP(X, Y, n_iter=500)
#mf_dgp_fix_lf_mean.optimize()

In [30]:
#y_mean_dgp, y_var_dgp = mf_dgp_fix_lf_mean.predict(x_test)

In [31]:
#print(calculate_metrics(y_test, y_mean_dgp, y_var_dgp))

In [32]:
#plt.scatter(y_mean_dgp,y_test)
#plt.plot([0, 1], [0, 1], '--k', transform=plt.gca().transAxes)
#plt.show()

In [33]:
X_currin_cheap = X[0]
X_currin_expensive = X[1]

y_currin_cheap = Y[0]
y_currin_expensive = Y[1]

X_borehole_expensive = Xb[1]
y_borehole_expensive = Yb[1]

import GPy
#from GPy.kern import Kern
#from GPy import Param, Model

m4 = GPy.models.GPRegression(X_currin_expensive, y_currin_expensive)
m4.optimize()

In [34]:
m4.optimize_restarts(num_restarts = 10)

Optimization restart 1/10, f = 15.053429496650764
Optimization restart 2/10, f = 15.053430386882848
Optimization restart 3/10, f = 15.053429677144344
Optimization restart 4/10, f = 15.053429537606393
Optimization restart 5/10, f = 15.053429467863873
Optimization restart 6/10, f = 15.053429508051218
Optimization restart 7/10, f = 15.053439987910705
Optimization restart 8/10, f = 15.05342979777585
Optimization restart 9/10, f = 15.053429659996672
Optimization restart 10/10, f = 15.05342966640193


In [35]:
mb = GPy.models.GPRegression(X_borehole_expensive, y_borehole_expensive)

In [36]:
mb.optimize()
mb.optimize_restarts(num_restarts = 10)

Optimization restart 1/10, f = 3.6495054933040394
Optimization restart 2/10, f = 6.723548248191144
Optimization restart 3/10, f = 3.6495069335503585
Optimization restart 4/10, f = 6.7235482482272255
Optimization restart 5/10, f = 3.649504768305629
Optimization restart 6/10, f = 6.723548212782571
Optimization restart 7/10, f = 3.6495056411696956
Optimization restart 8/10, f = 3.649504954478221
Optimization restart 9/10, f = 6.72354824819115
Optimization restart 10/10, f = 6.7235482481984725


In [37]:
X_test = x_test[:,0:4]

In [38]:
pred_mu_new, new_var = m4.predict(X_test)

In [39]:
print(calculate_metrics(y_test, pred_mu_new, new_var)) 

{'r2': 0.8798918432858639, 'rmse': 1.689328180573288, 'mnll': 1.8596885083540466}


In [40]:
xb_test = x_test_b[:,0:8]

b_mean, b_var = mb.predict(xb_test)

In [41]:
print(calculate_metrics(y_test_b, b_mean, b_var)) 

{'r2': 0.28151473444460606, 'rmse': 0.39227180257661104, 'mnll': 0.5629753937182819}
